![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

### An Introduction to the Iceberg Java API

#### [Part 1](https://tabular.io/blog/java-api-part-1/)

In [ ]:
import org.apache.iceberg.catalog.Catalog;
import org.apache.hadoop.conf.Configuration;
import org.apache.iceberg.CatalogProperties;
import org.apache.iceberg.jdbc.JdbcCatalog;
import org.apache.iceberg.hadoop.HadoopFileIO;

Map<String, String> properties = new HashMap<>();

properties.put(CatalogProperties.CATALOG_IMPL, JdbcCatalog.class.getName());
properties.put(CatalogProperties.URI, "jdbc:postgresql://postgres:5432/demo_catalog");
properties.put(JdbcCatalog.PROPERTY_PREFIX + "user", "admin");
properties.put(JdbcCatalog.PROPERTY_PREFIX + "password", "password");
properties.put(CatalogProperties.WAREHOUSE_LOCATION, "/home/iceberg/warehouse");
properties.put(CatalogProperties.FILE_IO_IMPL, HadoopFileIO.class.getName());

JdbcCatalog catalog = new JdbcCatalog();
Configuration conf = new Configuration();
catalog.setConf(conf);
catalog.initialize("demo", properties);
catalog.name();

In [ ]:
import org.apache.iceberg.catalog.Namespace;
import org.apache.iceberg.catalog.TableIdentifier;

Namespace nyc = Namespace.of("nyc");
catalog.listTables(nyc)

In [ ]:
import org.apache.iceberg.Schema;
import org.apache.iceberg.types.Types;

Schema schema = new Schema(
      Types.NestedField.required(1, "level", Types.StringType.get()),
      Types.NestedField.required(2, "event_time", Types.TimestampType.withZone()),
      Types.NestedField.required(3, "message", Types.StringType.get()),
      Types.NestedField.optional(4, "call_stack", Types.ListType.ofRequired(5, Types.StringType.get()))
    );
schema

In [ ]:
import org.apache.iceberg.PartitionSpec;

PartitionSpec spec = PartitionSpec.builderFor(schema)
      .hour("event_time")
      .identity("level")
      .build();
spec

In [ ]:
import org.apache.iceberg.catalog.TableIdentifier;

TableIdentifier name = TableIdentifier.of(nyc, "logs");
name

In [ ]:
catalog.createTable(name, schema, spec)

In [ ]:
catalog.dropTable(name)